## Test the `moviepy` library

Docs: https://zulko.github.io/moviepy/getting_started

In [3]:
import moviepy.editor as mpe
import moviepy as mp
import json
from tqdm import tqdm
import os
import numpy as np

In [1]:
part = 2
note_clip_dir = f"./data/recordings/violin_{part}/text"
sequence_dir = f"./data/sequence/bach_double/violin_{part}.txt"
output_path = f"./output/violin_{part}.mp4"

In [2]:
# Grab note names
with open("../data/note_names.txt") as fin:
    items = [line.split(",") for line in fin.read().strip().split("\n")]
    name2code = {name: int(code) for name, code in items}
    code2name = {int(code): name for name, code in items}

note_names = [name for name, _ in items]

In [4]:
# Load all clips
# Takes ~30 seconds
note2clip = {}

In [5]:
for note in tqdm(note_names, ncols=100):
    note2clip[note] = mpe.VideoFileClip(os.path.join(note_clip_dir, f"{note}.mp4"))

100%|███████████████████████████████████████████████████████████████| 33/33 [00:11<00:00,  2.80it/s]


In [6]:
# Chromatic stitching
min_duration = min([note2clip[note].duration for note in note_names])
print(f"Min duration:", min_duration)

chromatic_clips = []
for i, note in enumerate(note_names):
    # Accelerando!
    if i <= 4:
        duration = 1
    elif i <= 8:
        duration = 3/4
    elif i <= 12:
        duration = 1/2
    elif i < len(note_names) - 1:
        duration = np.interp(i, [11, len(note_names) - 1], [1/2, 1/8])
    else:
        duration = 2.4

    chromatic_clips.append(
      note2clip[note].subclip(0, duration)
        .fx(mp.audio.fx.all.audio_fadeout, 1/8 * 0.5)
    )
concat_clips = mp.editor.concatenate_videoclips(chromatic_clips)

# ~15 seconds
concat_clips.write_videofile(f"./output/chromatic_violin_{part}.mp4")

Min duration: 2.35
Moviepy - Building video ./output/chromatic_violin_2.mp4.
MoviePy - Writing audio in chromatic_violin_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./output/chromatic_violin_2.mp4



Moviepy - Done !
Moviepy - video ready ./output/chromatic_violin_2.mp4


In [7]:
# Grab piece data
notes = []
with open(sequence_dir) as fin:
    notes = [tuple(map(int, line.split(","))) for line in fin.read().strip().split("\n")]

## Render video

In [8]:
# Woohoo render the video
# https://stackoverflow.com/questions/28758474/how-to-concatenate-videos-in-moviepy

all_clips = []
factor = 0.15

for timestamp, code, duration in notes:
    all_clips.append(
        note2clip[code2name[code]].subclip(0, duration * factor)
            .fx(mp.audio.fx.all.audio_fadeout, factor * 0.25)
    )

# ~50 seconds
concat_clips = mp.editor.concatenate_videoclips(all_clips)

In [9]:
# ~30 seconds for building
# ~5-6 minutes for writing
concat_clips.write_videofile(output_path)

Moviepy - Building video ./output/violin_2.mp4.
MoviePy - Writing audio in violin_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./output/violin_2.mp4



Moviepy - Done !
Moviepy - video ready ./output/violin_2.mp4
